<a href="https://colab.research.google.com/github/Tiger-A/Yandex_Data-Science/blob/master/Pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Источник:

https://data-in.ru/data-catalog/datasets/187/#dataset-codebook

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Для эксперимента возьмём датасет "Крупные города России: объединенные данные по основным социально-экономическим показателям за 1985-2019 гг." инфраструктуры научно-исследовательских данных (https://data-in.ru/data-catalog/datasets/187/#dataset-codebook). Использовать будем только столбцы 'region' (субъект РФ), 'municipality' (муниципальное образование), 'year' (год), 'birth' (число родившихся на 1000 человек населения), 'wage' (cреднемесячная номинальная начисленная заработная плата, руб.). Сразу оговорюсь, что Москва и Санкт-Петербург являются отдельными субъектами Российской Федерации и в этом датафрейме отсутствуют.

In [ ]:
df = pd.read_csv('Krupnie_goroda-RF_1985-2019_187_09.12.21/data.csv', sep=';')
df = df[['region', 'municipality', 'year', 'birth', 'wage']]
df.sample(7)

,region,municipality,year,birth,wage
826,Вологодская область,Город Вологда,2014,14.4,30812.7
3864,Республика Татарстан,Казань,2011,14.2,21901.0
1138,Тверская область,Город Тверь,2015,12.5,31659.0
505,Архангельская область,Архангельск,2002,10.3,5381.0
3334,Республика Башкортостан,Уфа,2004,10.8,7859.8
3053,Ульяновская область,Ульяновск,2005,8.8,6702.8
1196,Камчатский край,Петропавловск-Камчатский городской округ,1999,8.7,3624.0


Минимальные данные можно получить использовав метод .describe(include = 'all'). Мы видим что у нас 4109 строки, по 81 региону и 202 городам. Средняя рождаемость на 1000 человек 11,39, минимальная - 3,4, максимальная - 36,1.

In [ ]:
df.describe(include = 'all')

,region,municipality,year,birth,wage
count,4109,4109,4109.000000,4104.000000,4098.000000
unique,81,202,NaN,NaN,NaN
top,Московская область,Барнаул,NaN,NaN,NaN
freq,365,27,NaN,NaN,NaN
mean,NaN,NaN,2007.474081,11.396564,19619.858897
std,NaN,NaN,7.686001,2.952200,18274.629869
min,NaN,NaN,1985.000000,3.400000,0.116000
25%,NaN,NaN,2002.000000,9.300000,4387.175000
50%,NaN,NaN,2008.000000,10.900000,15586.550000
75%,NaN,NaN,2014.000000,13.000000,30025.650000


## Агрегирование
Если объяснять простыми словами, то агрегирование - это процесс приведения некого большого массива значений к некому сжатому по определенному параметру массиву значений. Например, среднее, медиана, количество, сумма.

In [ ]:
df[['birth', 'wage']].agg(['mean', 'median', 'min', 'max'])

,birth,wage
mean,11.396564,19619.858897
median,10.900000,15586.550000
min,3.400000,0.116000
max,36.100000,170460.000000


Мы убедились, что среднее значение по рождаемости - 11,39, средняя зарплата - 19619. Таким образом мы провели агрегирование.

## Groupby
Но средняя температура по больнице нам не интересна, мы хотим знать победителей в лицо. Допустим нам нужно посмотреть средние значения с группировкой по городам и субъектам РФ. Для этого закономерно используем метод .groupby(['region', 'municipality']).agg('mean').

In [ ]:
df_groupby = df.groupby(['region', 'municipality']).agg('mean')
df_groupby.head(7)

year      birth          wage
region                municipality                                         
Алтайский край        Барнаул          2005.555556  10.881481  12700.674407
                      Бийск            2009.500000  11.000000  13479.875000
                      Рубцовск         2009.500000   9.680000  12774.935000
Амурская область      Благовещенск     2005.555556  11.803704  17359.895556
Архангельская область Архангельск      2005.555556  10.477778  18911.154519
                      Северодвинск     2009.500000  10.390000  27084.675000
Астраханская область  Город Астрахань  2005.555556  12.100000  13896.580111

Но мы на этом не успкаиваемся и хотим больше данных: среднее, медиану, минимум, максимум.

In [ ]:
agg_func_math = {
    'birth': ['mean', 'median', 'min', 'max'],
    'wage': ['mean', 'median', 'min', 'max']
}

In [ ]:
df.groupby(['region', 'municipality']).agg(agg_func_math).head(7)

birth                    \
                                            mean median  min   max   
region                municipality                                   
Алтайский край        Барнаул          10.881481  10.70  7.6  16.0   
                      Бийск            11.000000  10.70  8.7  13.5   
                      Рубцовск          9.680000   9.65  7.2  12.3   
Амурская область      Благовещенск     11.803704  11.80  9.3  16.1   
Архангельская область Архангельск      10.477778  10.70  7.9  15.4   
                      Северодвинск     10.390000  10.40  7.0  13.1   
Астраханская область  Город Астрахань  12.100000  12.30  8.9  14.5   

                                               wage                      \
                                               mean    median       min   
region                municipality                                        
Алтайский край        Барнаул          12700.674407   7696.00     0.213   
                      Бийск            13479.875000  12575.75  1386.000   
                      Рубцовск         12774.935000  11907.25  1192.000   
Амурская область      Благовещенск     17359.895556  11913.70     0.219   
Архангельская область Архангельск      18911.154519  13432.00     0.236   
                      Северодвинск     27084.675000  23156.80  2407.000   
Астраханская область  Город Астрахань  13896.580111   8448.80     0.170   

                                                
                                           max  
region                municipality              
Алтайский край        Барнаул          37855.1  
                      Бийск            30203.9  
                      Рубцовск         29941.9  
Амурская область      Благовещенск     50675.5  
Архангельская область Архангельск      54598.8  
                      Северодвинск     63519.0  
Астраханская область  Город Астрахань  39924.8

Посмотрим топ городов по зарплатам.

In [ ]:
df.groupby(['region', 'municipality']).agg('mean').sort_values(by='wage', ascending=False).head(7)

year  \
region                                   municipality                 
Московская область                       Долгопрудный   2017.000000   
Ямало-Ненецкий автономный округ          Новый Уренгой  2010.277778   
Московская область                       Реутов         2017.000000   
                                         Домодедово     2014.500000   
Ханты-Мансийский автономный округ - Югра Сургут         2009.500000   
Красноярский край                        Норильск       2009.500000   
Ямало-Ненецкий автономный округ          Ноябрьск       2010.277778   

                                                            birth  \
region                                   municipality               
Московская область                       Долгопрудный   10.420000   
Ямало-Ненецкий автономный округ          Новый Уренгой  13.894444   
Московская область                       Реутов          9.360000   
                                         Домодедово     13.290000   
Ханты-Мансийский автономный округ - Югра Сургут         16.595000   
Красноярский край                        Норильск       12.700000   
Ямало-Ненецкий автономный округ          Ноябрьск       13.400000   

                                                                wage  
region                                   municipality                 
Московская область                       Долгопрудный   69366.200000  
Ямало-Ненецкий автономный округ          Новый Уренгой  64519.627778  
Московская область                       Реутов         53252.200000  
                                         Домодедово     52964.010000  
Ханты-Мансийский автономный округ - Югра Сургут         51231.465000  
Красноярский край                        Норильск       50389.215000  
Ямало-Ненецкий автономный округ          Ноябрьск       50348.477778

А что если посмотреть данные в разрезе по годам...?

## Pivot table

Ответим на этот вопрос чуть позже, а пока создадим сводную таблицу аналогичную той, что мы уже делали ранее.

In [ ]:
df_pivot_table = df.pivot_table(index=['region', 'municipality'])
df_pivot_table.head(7)

birth          wage         year
region                municipality                                         
Алтайский край        Барнаул          10.881481  12700.674407  2005.555556
                      Бийск            11.000000  13479.875000  2009.500000
                      Рубцовск          9.680000  12774.935000  2009.500000
Амурская область      Благовещенск     11.803704  17359.895556  2005.555556
Архангельская область Архангельск      10.477778  18911.154519  2005.555556
                      Северодвинск     10.390000  27084.675000  2009.500000
Астраханская область  Город Астрахань  12.100000  13896.580111  2005.555556

Уже на этом этапе видно, что сводная таблица достаточно умная и сама агрегировала данные и посчитала средние значения.

Но если мы захотим расширить таблицу новыми значениями - медианой, то увидим, что конечный результат по структуре будет отличаться от того, что мы делали при использовании метода groupby. 

In [ ]:
df_pivot_table = df.pivot_table(index=['region', 'municipality'],
                                values=['birth', 'wage'],
                                aggfunc=[np.mean, np.median])
df_pivot_table.head(7)

mean               median  \
                                           birth          wage  birth   
region                municipality                                      
Алтайский край        Барнаул          10.881481  12700.674407  10.70   
                      Бийск            11.000000  13479.875000  10.70   
                      Рубцовск          9.680000  12774.935000   9.65   
Амурская область      Благовещенск     11.803704  17359.895556  11.80   
Архангельская область Архангельск      10.477778  18911.154519  10.70   
                      Северодвинск     10.390000  27084.675000  10.40   
Астраханская область  Город Астрахань  12.100000  13896.580111  12.30   

                                                 
                                           wage  
region                municipality               
Алтайский край        Барнаул           7696.00  
                      Бийск            12575.75  
                      Рубцовск         11907.25  
Амурская область      Благовещенск     11913.70  
Архангельская область Архангельск      13432.00  
                      Северодвинск     23156.80  
Астраханская область  Город Астрахань   8448.80

Так мы плавно подошли к тем преимуществам, которые делает сводные таблицы швейцарским ножом для аналитиков. Если использую groupby мы "укрупняли" строки по городам, то сейчас мы можем "развернуть" столбец, например, 'year' (год) и посмотреть данные в разрезе по годам.   

In [ ]:
df_pivot_table = df.pivot_table(index=['region', 'municipality'],
                                values='birth',
                                columns='year')
df_pivot_table.head(7)

birth                                   \
year                                   1985  1990 1991  1995  1996 1997 1998   
region                municipality                                             
Алтайский край        Барнаул          16.0  12.1  NaN   7.9   7.6  7.6  8.2   
                      Бийск             NaN   NaN  NaN   NaN   NaN  NaN  NaN   
                      Рубцовск          NaN   NaN  NaN   NaN   NaN  NaN  NaN   
Амурская область      Благовещенск     16.1  15.4  NaN  10.0  10.0  9.3  9.3   
Архангельская область Архангельск      15.4  12.1  NaN   8.6   8.8  8.3  8.9   
                      Северодвинск      NaN   NaN  NaN   NaN   NaN  NaN  NaN   
Астраханская область  Город Астрахань  13.9  13.2  NaN   8.9   8.9  9.0  9.0   

                                                       ...                    \
year                                  1999 2000  2001  ...  2010  2011  2012   
region                municipality                     ...                     
Алтайский край        Барнаул          8.1  8.7   9.2  ...  12.1  11.8  12.8   
                      Бийск            NaN  8.7   9.2  ...  12.1  12.1  13.5   
                      Рубцовск         NaN  8.5   8.5  ...  10.8  11.3  12.3   
Амурская область      Благовещенск     9.3  9.8  10.0  ...  13.2  12.1  13.7   
Архангельская область Архангельск      7.9  8.3   9.4  ...  11.7  11.1  11.9   
                      Северодвинск     NaN  7.0   7.8  ...  11.3  11.3  11.9   
Астраханская область  Город Астрахань  9.3  9.9  10.5  ...  13.8  13.1  14.2   

                                                                           \
year                                   2013  2014  2015  2016  2017  2018   
region                municipality                                          
Алтайский край        Барнаул          12.9  12.9  13.7  13.3  11.9  10.8   
                      Бийск            13.3  12.7  13.1  11.6   9.8   9.3   
                      Рубцовск         11.6  11.2  10.7   9.7   8.2   7.8   
Амурская область      Благовещенск     12.7  12.3  13.9  13.6  12.7  11.8   
Архангельская область Архангельск      11.8  12.1  11.8  11.6  10.0   9.1   
                      Северодвинск     11.9  11.1  13.1  11.6  10.8   9.8   
Астраханская область  Город Астрахань  13.9  14.1  14.5  14.5  12.4  11.9   

                                             
year                                   2019  
region                municipality           
Алтайский край        Барнаул           9.8  
                      Бийск             8.9  
                      Рубцовск          7.2  
Амурская область      Благовещенск     10.6  
Архангельская область Архангельск       8.1  
                      Северодвинск      9.3  
Астраханская область  Город Астрахань  11.0  

[7 rows x 28 columns]

И, конечно, данные в сводной таблице можно фильтровать. Создадим сводную таблицу и оставим в ней данные по городам, в которых рождаемость на 1000 человек превышает 12, зарплата выше 40.000 и отсортируем всё по убыванию рождаемости.

In [ ]:
df2 = df.pivot_table(index=['region', 'municipality'])
df2.query("`birth`>12 and `wage`>40000").sort_values(by='birth', ascending=False)

birth  \
region                                   municipality               
Ханты-Мансийский автономный округ - Югра Сургут         16.595000   
Ямало-Ненецкий автономный округ          Салехард       15.896296   
Московская область                       Раменский      14.922222   
Ямало-Ненецкий автономный округ          Новый Уренгой  13.894444   
                                         Ноябрьск       13.400000   
Московская область                       Домодедово     13.290000   
Ханты-Мансийский автономный округ - Югра Нефтеюганск    13.250000   
Московская область                       Красногорский  13.141667   
Чукотский автономный округ               Анадырь        12.825926   
Красноярский край                        Норильск       12.700000   

                                                                wage  \
region                                   municipality                  
Ханты-Мансийский автономный округ - Югра Сургут         51231.465000   
Ямало-Ненецкий автономный округ          Салехард       40139.665926   
Московская область                       Раменский      42697.511111   
Ямало-Ненецкий автономный округ          Новый Уренгой  64519.627778   
                                         Ноябрьск       50348.477778   
Московская область                       Домодедово     52964.010000   
Ханты-Мансийский автономный округ - Югра Нефтеюганск    40221.240000   
Московская область                       Красногорский  47492.783333   
Чукотский автономный округ               Анадырь        45773.736815   
Красноярский край                        Норильск       50389.215000   

                                                               year  
region                                   municipality                
Ханты-Мансийский автономный округ - Югра Сургут         2009.500000  
Ямало-Ненецкий автономный округ          Салехард       2005.555556  
Московская область                       Раменский      2015.000000  
Ямало-Ненецкий автономный округ          Новый Уренгой  2010.277778  
                                         Ноябрьск       2010.277778  
Московская область                       Домодедово     2014.500000  
Ханты-Мансийский автономный округ - Югра Нефтеюганск    2009.500000  
Московская область                       Красногорский  2013.500000  
Чукотский автономный округ               Анадырь        2005.555556  
Красноярский край                        Норильск       2009.500000

## Pivot

Всё было замечательно, но в Pandas кроме pivot_table есть ещё просто pivot. Посмотрим что это за зверь и чем они отличаются. 

Создадим pivot: рождаемость в разрезе по регионам и годам.

In [ ]:
df_pivot = df.pivot(index='region',
                    values='birth',
                    columns='year')
df_pivot.head(7)

ValueError: ignored

Мы получили ошибку "Index contains duplicate entries, cannot reshape". Что-то не так с индексами, попробуем создать pivot с индексами по городам, а не регионам.

In [ ]:
df_pivot = df.pivot(index='municipality',
                    values='birth',
                    columns='year')
df_pivot.head(7)

year,1985,1990,1991,1995,1996,1997,1998,1999,2000,2001,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
municipality,,,,,,,,,,,,,,,,,,,,,
Абакан,16.8,14.4,NaN,10.4,9.7,9.2,9.3,9.0,10.1,10.0,...,14.6,14.6,14.9,15.4,14.9,15.4,14.5,13.5,12.2,11.5
Альметьевский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.9,10.7,...,13.9,13.9,14.8,15.8,15.8,17.4,16.7,15.0,13.6,12.7
Анадырь,15.8,13.5,NaN,9.5,7.8,9.1,12.0,11.0,10.5,11.4,...,17.9,14.0,13.5,12.2,12.1,12.6,12.2,11.7,11.3,8.9
Ангарский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,9.1,...,11.9,12.6,13.3,13.2,13.2,14.3,13.6,11.7,10.8,9.4
Арзамас,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,9.0,...,11.0,10.9,12.1,11.5,11.2,13.1,12.9,11.3,10.3,9.6
Артёмовский,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,8.6,...,12.1,12.7,13.3,12.9,13.2,13.3,12.1,11.8,11.3,10.6
Архангельск,15.4,12.1,NaN,8.6,8.8,8.3,8.9,7.9,8.3,9.4,...,11.7,11.1,11.9,11.8,12.1,11.8,11.6,10.0,9.1,8.1


Всё получилось. Как мы уже заметили всё дело в индексах. На самом деле всё в повторяющихся индексах. Как мы видили ранее в датафрейме есть позиции с несколькими городами в рамках одного субъекта, так получаются дублированные данные индексов с которыми не умеет работать pivot.

## Вывод:
Groupby, pivot и pivot_table удобные и привычные инструменты для работы с данными. Groupby позволяет кодом в одну строку получить агрегированные и сортированные данные, а pivot и pivot_table работать в более глубоком разрезе. Pivot_table предпочтителен, т.к. не ограничивает вас в уникальности значений в столбце индекса. И, конечно, все эти данные можно фильтровать под ваши запросы.
